In [1]:
import pandas as pd
import numpy as np
import random
from glob import glob
import os
import re

## Generate simulation pmats

In [1]:
def generate_gt_pmat(n_rows, clones, samples, n_mutations, high_prob, low_prob): #n_rows, clones, samples, n_mutations, high_prob, low_prob
    c = [f'c{w+1}' for w in range(clones)]
    s = [f's{w+1}' for w in range(samples)]
    mux = pd.MultiIndex.from_product([c, s])
    
    #Generate a dataframe with with default value
    gt = pd.DataFrame(low_prob, index=np.arange(n_rows) // n_mutations, columns=mux)
    
    # random assign high probability for mutation by random index with no repeat
    for c, i in zip(gt.columns.levels[0], 
                    np.random.choice(np.unique(gt.index), clones, replace=False)):
        gt.loc[i, c] = high_prob
    
    gt.columns = gt.columns.map(lambda clones: f'{clones[0]}_{clones[1]}')
    gt = gt.reset_index(drop=True)
    
    gt.insert(loc=0, column='bulk', value=0.0001)
    #gt_size = gt.shape

    matrix_path = '../../data/simulated_matrix_output/no_dropout/0_cells_0_pos.csv'
    gt.to_csv(matrix_path, index=False, sep=' ', header= False)

    return gt


## should bulk be removed when adding noise?

# max is max len of pmat
def add_noise(gt, max):
    
    # x is n cells sharing a non-informative phylo signal
    for x in range(1, 8):
        gt_w_noise = gt.copy()
        counter = 0
        
        # i is the number of added non-informative positions
        for i in range(1, max):
            if len(gt_w_noise) != max:
                counter+=1
                # add new row with low probability for mutation
                gt_w_noise.loc[-1,:] = 0.0001

                # reset index
                gt_w_noise = gt_w_noise.reset_index(drop=True)

                # assign high probability to x randomly selected cells on last row
                gt_w_noise.iloc[-1:, random.sample(list(range(gt_w_noise.shape[1])), x)] = 0.9
                
                # n is number of added non-informative positions
                n = str(counter)
                
                matrix_path = '../../data/simulated_matrix_output/no_dropout/' + str(x) + '_cells_' + n + '_pos.csv'
                gt_w_noise.to_csv(matrix_path, index=False, sep=' ', header= False)
                #print(len(gt_w_noise))


In [2]:
n_rows = 6 #total number of mutations in pmat. This must be clones * n_mutations? #6
clones = 6 # number of clones #6
samples = 6 # number of cells per clone #6
n_mutations = 1 # number of mutations per clone
high_prob = 0.9 # probability for presence of mutation
low_prob = 0.0001 # probability for absence of mutation

max_mut = 18 # max total len of pmat. max_mut - n_rows = number of added noise pos
max_samples = 8 # 

In [3]:
gt = generate_gt_pmat(n_rows, clones, samples, n_mutations, high_prob, low_prob)
gt

NameError: name 'pd' is not defined

In [25]:
add_noise(gt, max_mut)

In [21]:
len(gt)

6